# Paper Keywords Generation

## Introduction
This notebook aims to generate keywords for scientific papers that posted online as PDF files. We do this by

1. <b>Getting Data</b> - we collect a url, title and abstract of papers from our database.
2. <b>Keywords Generation</b>  
    
  1. We read in content of online pdf file, then count occurrence of each word in our pre-listed keywords list and select 5 highest words.
  2. In case that there are less than 5 pre-listed keywords are found in the file, we clean up the paper's title and abstract and using YAKE to generate the rest.

## 1. Getting Data

In [ ]:
# Connect with database
from pymongo import MongoClient

DB_NAME = ''
COLLECTION_NAME = ''

mongo_uri = "".format(DB_NAME)
client = MongoClient(mongo_uri)

# Database: TechVault
db = client[DB_NAME]

# Collection: blogs
collection = db[COLLECTION_NAME]

In [ ]:
# Read all papers

papers = []

for doc in collection.find():
    if doc['type'] == 'paper':
        # collect url, title and abstract
        paper = {'url': doc['link'],
                'title': doc['title'],
                'abstract': doc['abstract']}
        papers.append(paper)

papers[0]

In [ ]:
len(papers)

## 2. Keywords Generation
### 2.1 Keywords Matching

In [ ]:
# Dictionary of keywords
# Key: Searching words
# Value: Displayed words

keywords = {"Machine Learning": "Machine Learning",
            "Supervised Learning": "Supervised Learning",
            "Unsupervised Learning": "Unsupervised Learning",
            "Multilabel Classification": "Multilabel Classification",
            "Clustering": "Clustering",
            "K-Means": "K-Means",
            "DBSCAN": "DBSCAN",
            "Hierarchical Clustering": "Hierarchical Clustering",
            "Deep Learning": "Deep Learning",
            "Data Mining": "Data Mining",
            "Linear regression": "Linear regression",
            "Logistic regression": "Logistic regression",
            "SVM": "SVM",
            "Natural Language Processing": "Natural Language Processing",
            "Computer Vision": "Computer Vision",
            "KNN": "KNN",
            "Random forest": "Random forest",
            "Decision Tree": "Decision Tree",
            "Regularization": "Regularization",
            "Ensemble Learning": "Ensemble Learning",
            "Gradient Boosting": "Gradient Boosting",
            "Feature Selection": "Feature Selection",
            "Reinforcement Learning": "Reinforcement Learning",
            "Virtual Reality": "Virtual Reality",
            "Augmented reality": "Augmented reality",
            "Autonomous driving": "Autonomous driving",
            "Optics": "Optics",
            "Biology": "Biology",
            "C++": "C++",
            "Java": "Java",
            "Python": "Python",
            "React JS": "React JS",
            "Computer Network": "Computer Networks",  # remove s
            "Frontend": "Frontend",
            "Backend": "Backend",
            "High Scalability": "High Scalability",
            "Cloud computing": "Cloud computing",
            "Parallel Computing": "Parallel Computing",
            "CUDA": "CUDA",
            "Distributed System": "Distributed Systems",  # remove s
            "Apache ZooKeeper": "Apache ZooKeeper",
            "Streaming analytic": "Streaming analytics",
            "Model Selection": "Model Selection",
            "Model Evaluation": "Model Evaluation",
            "Apache Kafka": "Apache Kafka",
            "HDFS": "HDFS",
            "Amazon S3": "Amazon S3",
            "Pub-Sub": "Pub-Sub",
            "Leader Election": "Leader Election",
            "Clock Synchronization": "Clock Synchronization",
            "Graph": "Graphs",  # remove s
            "Information Retrieval": "Information Retrieval",
            "SQL": "SQL",
            "Graph Database": "Graph Database",
            "Database Management": "Database Management",
            "Storage": "Storage",
            "Memor": "Memory",
            "Garbage Collection": "Garbage Collection",
            "Map-Reduce": "Map-Reduce",
            "Network Protocol": "Network Protocols",  # remove s
            "Cyber Security": "Cyber Security",
            "Assembly Language": "Assembly Language",
            "Computational Complexity Theor": "Computational Complexity Theory",
            "Computer Architecture": "Computer Architecture",
            "Human-Computer Interface": "Human-Computer Interface",
            "Data Structure": "Data Structures",  # remove s
            "Discrete Mathematic": "Discrete Mathematics",
            "Hacking": "Hacking",
            "Quantum Computing": "Quantum Computing",
            "Robotic": "Robotics",  # remove s
            "Engineering Practice": "Engineering Practices",  # remove s
            "Software Tool": "Software Tools",  # remove s
            "Mathematical Logic": "Mathematical Logic",
            "Graph Theor": "Graph Theory",
            "Computational Geometr": "Computational Geometry",
            "Compiler": "Compilers",  # remove s
            "Distributed Computing": "Distributed Computing",
            "Software Engineering": "Software Engineering",
            "Bioinformatic": "Bioinformatics",  # remove s
            "Computational Chemistry": "Computational Chemistry",
            "Computational Neuroscience": "Computational Neuroscience",
            "Computational physics": "Computational physics",
            "Numerical algorithm": "Numerical algorithms",  # remove s
            "JavaScript": "JavaScript",
            "HTML": "HTML",
            "Web Development": "Web Development",
            "App Development": "App Development",
            "CSS": "CSS",
            "PHP": "PHP",
            "BlockChain": "BlockChain",
            "Hardware": "Hardware",
            "VLSI": "VLSI",
            "Cluster Computing": "Cluster Computing",
            "Kubernetes": "Kubernetes",
            "Go": "Go-Lang",
            "File System": "File Systems",  # remove s
            "Statistic": "Statistics",  # remove s
            "Optimization": "Optimization",
            "Knowledge Graph": "Knowledge Graph",
            "RNN": "RNN",
            "CNN": "CNN",
            "Physical Design": "Physical Design",
            "Memory management": "Memory management",
            "PCA": "PCA",
            "LDA": "LDA",
            "Feature Engineering": "Feature Engineering",
            "Data manipulation": "Data manipulation",
            "ACID": "ACID",
            "BASE": "BASE",
            "Consistency": "Consistency",
            "Disaster recovery": "Disaster recovery",
            "Replication": "Replication",
            "Fault tolerance": "Fault tolerance",
            "Deployment": "Deployment",
            "Processor": "Processors",  # remove s
            "Multi-Threading": "Multi-Threading",
            "Queue": "Queue",
            "Stack": "Stack",
            "Dynamic Programming": "Dynamic Programming",
            "Graph Traversal": "Graph Traversal",
            "Device": "Devices",  # remove s
            "Data analysis": "Data analysis",
            "Probability": "Probability",
            "Mathematic": "Mathematics",  # remove s
            "Genomic": "Genomics",  # remove s
            "Data Infrastructure": "Data Infrastructure",
            "Software Principles and Practices": "Software Principles and Practices",
            "Image Processing": "Image Processing",
            "Audio Processing": "Audio Processing",
            "Signal Processing": "Signal Processing",
            "Pattern Recognition": "Pattern Recognition",
            "Computation and Language": "Computation and Language",
            "Artificial Intelligence": "Artificial Intelligence",
            "Computation and Language": "Computation and Language",
            "Computational Complexit": "Computational Complexity",
            "Computational Engineering": "Computational Engineering",
            "Finance": "Finance",  # remove "and Science" from "Finance, and Science"
            "Computational Geometry": "Computational Geometry",
            "Game Theory": "Game Theory",  # remove "Computer Science" from "Computer Science and Game Theory"
            "Computer Vision": "Computer Vision",  # break down from "Computer Vision and Pattern Recognition"
            "Pattern Recognition": "Pattern Recognition",  # break down from "Computer Vision and Pattern Recognition"
            "Computers and Society": "Computers and Society",
            "Cryptography and Security": "Cryptography and Security",
            "Data Structure": "Data Structures",  # break down from "Data Structures and Algorithms"
            "Algorithm": "Algorithms",  # break down from "Data Structures and Algorithms"
            "Database": "Databases",  # break down from "Databases; Digital Libraries"
            "Digital Librar": "Digital Libraries",  # break down from "Databases; Digital Libraries"
            "Distributed Computing": "Distributed Computing",  # break down from "Distributed, Parallel, and Cluster Computing"
            "Parallel Computing": "Parallel Computing",  # break down from "Distributed, Parallel, and Cluster Computing"
            "Cluster Computing": "Cluster Computing",  # break down from "Distributed, Parallel, and Cluster Computing"
            "Emerging Technolog": "Emerging Technologies",
            "Formal Language": "Formal Languages",  # break down from "Formal Languages and Automata Theory"
            "Automata Theory": "Automata Theory",  # break down from "Formal Languages and Automata Theory"
            "General Literature": "General Literature",
            "Graphic": "Graphics",  # remove s
            "Human-Computer Interaction": "Human-Computer Interaction",
            "Information Theory": "Information Theory",
            "Logic in Computer Science": "Logic in Computer Science",
            "Mathematical Software": "Mathematical Software",
            "Multiagent System": "Multi-agent Systems",  # remove s from "Systems"
            "Multi-agent System": "Multi-agent Systems",  # remove s from "Systems" and add -
            "Multimedia": "Multimedia",
            "Networking and Internet Architecture": "Networking and Internet Architecture",
            "Neural and Evolutionary Computing": "Neural and Evolutionary Computing",
            "Numerical Analysis": "Numerical Analysis",
            "Operating System": "Operating Systems",  # remove s from "Systems"
            "Performance": "Performance",
            "Programming Language": "Programming Languages",  # remove s 
            "Social and Information Networks": "Social and Information Networks",
            "Software Engineering": "Software Engineering",
            "Sound": "Sound",
            "Symbolic Computation": "Symbolic Computation",
            "Systems and Control": "Systems and Control"
           }

In [ ]:
def countKeywords(text, keywords):
    ''' Count occurence of keywords in the text, return a dict of words and its occurence'''
    d = {}
    text = ' ' + text + ' '
    # Abbreviations list
    abbreviations = ["SVM", "KNN", "CUDA", "HDFS", "SQL", "HTML", "CSS", "PHP",
                    "VLSI", "RNN", "CNN", "PCA", "LDA", "ACID", "BASE"]
    
    for search_word, display_word in keywords.items():
        # 'Go' can be a sub-string of many words, to be precise, we'll search for the word " Go "
        if search_word == "Go":
            search_word = ' ' + search_word + ' '
        
        # Lower the word if it's not an abbreviation
        elif search_word not in abbreviations:
            search_word = search_word.lower()
        
        # Count occurence of searching word (case sensitive)
        oc = text.count(search_word)
        
        # Append to the dictionary if the word occurs 1 or more time
        if oc > 0:
            d[display_word] = oc

    return d

In [ ]:
# Install pdfPlumber package
# pip install pdfplumber

In [ ]:
import pdfplumber
import io
import requests
import yake

def readPDF(url):
    ''' 
    Read PDF file from a URL
    Return a text string
    '''
    r = requests.get(url)
    f = io.BytesIO(r.content)
    text = "" # returned text
    try:
        # Open pdf file
        with pdfplumber.open(f) as pdf:
            pages = pdf.pages
            # Iterate through pdf document pages
            for page in pages:
                # Concat page content to text if there are any
                if page.extract_text():
                    text += '\n' + page.extract_text()

        return text
    except:
        print("PDF file is not found!")
        return ""
    

def keywordsFromPDF(url, keywords):
    '''
    Extracts keywords from a pdf document
      String url: a url of a pdf file
      Dict keywords: a dictionary of searched word and displayed word
    Return
      A list of five words that have highest occurences, can be less than five words
    '''
    N_KEYWORDS = 5
    text = readPDF(url).lower()
        
    occ = countKeywords(text, keywords)

    # get a list of top 5 words
    result = list(dict(sorted(occ.items(), key=lambda x: x[1], reverse=True)).keys())[:N_KEYWORDS]
    
    return result

In [ ]:
# Functions for data cleansing
from bs4 import BeautifulSoup
import re
import string
import copy

# Funtion to remove HTML tags
def removeHTMLTags(text):
    return BeautifulSoup(text, 'html.parser').get_text()
    
    
# Function to remove more special characters and escape characters
def removeExtraWhitespaceEsc(text):
    #pattern = r'^\s+$|\s+$'
    pat = r'^\s*|\s\s*'
    return re.sub(pat, ' ', text).strip()
    
    
# Function to remove commas and periods
def removeCommasPeriods(text):
    pat = r'[.,]+'
    return re.sub(pat, '', text)


# Function to remove words that include special character
def removeSpecialCharacterWords(text):
    # define the pattern to keep only letters, numbers, dash and white spaces
    pat = r'[a-zA-Z0-9]*[^a-zA-Z0-9_\s]+[a-zA-Z0-9]*'
    return re.sub(pat, '', text)


def clean_data(text):
    '''
    Clean text
    '''
    clean_text = removeHTMLTags(text)
    clean_text = removeExtraWhitespaceEsc(clean_text)
    clean_text = removeCommasPeriods(clean_text)
    clean_text = removeSpecialCharacterWords(clean_text)
    
    return clean_text

In [ ]:
def keywordsFromYAKE(text, numOfKeywords):
    '''
    Extracts keywords from text by using YAKE
    '''
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 2  # max number of words in generated keywords
    deduplication_threshold = 0.1
    numOfKeywords = numOfKeywords
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    kws = custom_kw_extractor.extract_keywords(text)
    result = [x for x, y in kws]
    
    return result


def keywordsFromPaper(paper, keywords):
    '''
    Extracts keywords from a paper.
      Dict paper: a dictionary of the paper pdf url, title and abstract
      Dict keywords: a dictionary of searched word and displayed word
    Return
      A list of five words
    '''
    N_KEYWORDS = 5
    
    # Keywords from performing keywords matching
    result = keywordsFromPDF(paper['url'], keywords)
    print('kwm: {}'.format(result))
    
    # If the result has less than 5 keywords then use YAKE with title and abstract
    keys = ['title', 'abstract']
    
    for k in keys:
        if len(result) < N_KEYWORDS:
            n_left = N_KEYWORDS - len(result)
            yake_kws = keywordsFromYAKE(clean_data(paper[k]), n_left)
            print('{}: {}'.format(k, yake_kws))
            result += [x for x in yake_kws if x not in result]
    
    return result 

In [ ]:
# Generate keywords for all papers
for paper in papers:
    print(paper['url'])
    paper['keywords'] = keywordsFromPaper(paper, keywords)
    print()